In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import sqlite3

## Store CSV into DataFrame, Extract & Transform
**We have five csv file:**
* Airlines - has information about airline_id(IATA Code) and airline name;
* Airports - has information about airport_id, airport, city, state, country, latitude, longitude
* Flights  - contains information about date, day of the week,airline id, flight number,tail number,origin airport, destination airport, scheduled departure, actual departure time,delay(in minutes), scheduled arrival, Arrival delay, and cancellation.
* tweets   - information about tweets regarding the flight experience,ariline and date of tweets.
* there is one additional file: cancellation reason. 

# 1. Airline

In [2]:
airline_file = "./Resources/airlines.csv"
airline_df = pd.read_csv(airline_file)
airline_df

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways
5,OO,Skywest Airlines Inc.
6,AS,Alaska Airlines Inc.
7,NK,Spirit Air Lines
8,WN,Southwest Airlines Co.
9,DL,Delta Air Lines Inc.


In [3]:
airline_df.columns = ['AIRLINE_ID','AIRLINE']
# airline_df.set_index('AIRLINE_ID',inplace=True)
airline_df

,AIRLINE_ID,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways
5,OO,Skywest Airlines Inc.
6,AS,Alaska Airlines Inc.
7,NK,Spirit Air Lines
8,WN,Southwest Airlines Co.
9,DL,Delta Air Lines Inc.


# 2. Airport

In [4]:
airports_file = "./Resources/airports.csv"
airports_df = pd.read_csv(airports_file)
airports_df.columns =['AIRPORT_ID','AIRPORT','CITY','STATE','COUNTRY','LATITUDE','LONGITUDE']
# airports_df.set_index('AIRPORT_ID',inplace=True)
airports_df.head()

,AIRPORT_ID,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


# 3. Flight

In [5]:
flights_file = "./Resources/flights.csv"
flights_df = pd.read_csv(flights_file)
flights_df.head(5)

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,12,1,2,AA,1230,N3KSAA,SEA,DFW,5,...,538.0,-17.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,12,1,2,DL,1426,N820DN,SFO,MSP,5,...,536.0,-4.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,12,1,2,AA,2406,N850AA,LAX,DFW,10,...,453.0,-8.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,12,1,2,UA,680,N38473,SFO,ORD,14,...,602.0,-30.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,12,1,2,AA,883,N199UW,SFO,CLT,15,...,810.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


### Check Datatypes of all the columns

In [6]:
flights_df.dtypes

YEAR                     int64
MONTH                    int64
DAY                      int64
DAY_OF_WEEK              int64
AIRLINE                 object
FLIGHT_NUMBER            int64
TAIL_NUMBER             object
ORIGIN_AIRPORT          object
DESTINATION_AIRPORT     object
SCHEDULED_DEPARTURE      int64
DEPARTURE_TIME         float64
DEPARTURE_DELAY        float64
TAXI_OUT               float64
WHEELS_OFF             float64
SCHEDULED_TIME           int64
ELAPSED_TIME           float64
AIR_TIME               float64
DISTANCE                 int64
WHEELS_ON              float64
TAXI_IN                float64
SCHEDULED_ARRIVAL        int64
ARRIVAL_TIME           float64
ARRIVAL_DELAY          float64
DIVERTED                 int64
CANCELLED                int64
CANCELLATION_REASON     object
AIR_SYSTEM_DELAY       float64
SECURITY_DELAY         float64
AIRLINE_DELAY          float64
LATE_AIRCRAFT_DELAY    float64
WEATHER_DELAY          float64
dtype: object

## Subset the airlines matching the tweets data

In [7]:
flights_df = flights_df.loc[(flights_df['AIRLINE']=='UA')| (flights_df['AIRLINE']=='US')| (flights_df['AIRLINE']=='AA')
                            | (flights_df['AIRLINE']=='WN') | (flights_df['AIRLINE']=='VX')]

### Check NaN values

In [8]:
# Drop rows of NaN
# There isn't a row that's completely NaN
all_nan_df = flights_df[flights_df.isna().all(axis=1)]
all_nan_df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY


In [9]:
# However, for the flights that's not delayed/ cancelled, those columns are NaN
any_nan_df = flights_df[flights_df.isna().any(axis=1)]
any_nan_df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,12,1,2,AA,1230,N3KSAA,SEA,DFW,5,...,538.0,-17.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,12,1,2,AA,2406,N850AA,LAX,DFW,10,...,453.0,-8.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,12,1,2,UA,680,N38473,SFO,ORD,14,...,602.0,-30.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,12,1,2,AA,883,N199UW,SFO,CLT,15,...,810.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
5,2015,12,1,2,AA,2020,N977UY,PHX,CLT,15,...,529.0,-32.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# For readibility, we replace NaN with 0 
flights_df = flights_df.fillna(0)
flights_df[flights_df.isna().any(axis=1)]

# For excluding 1 column: any_nan_df.loc[:, any_nan_df.columns != 'CANCELLATION_REASON'].fillna(0)

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY


### Modify Columns and Labels of categories

In [11]:
# DROP DELAY columns
flights_df = flights_df.drop(columns=['AIR_SYSTEM_DELAY', 'SECURITY_DELAY','AIRLINE_DELAY',
                         'LATE_AIRCRAFT_DELAY','WEATHER_DELAY'])
flights_df

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON
0,2015,12,1,2,AA,1230,N3KSAA,SEA,DFW,5,...,194.0,1660,533.0,5.0,555,538.0,-17.0,0,0,0
2,2015,12,1,2,AA,2406,N850AA,LAX,DFW,10,...,141.0,1235,445.0,8.0,501,453.0,-8.0,0,0,0
3,2015,12,1,2,UA,680,N38473,SFO,ORD,14,...,216.0,1846,557.0,5.0,632,602.0,-30.0,0,0,0
4,2015,12,1,2,AA,883,N199UW,SFO,CLT,15,...,264.0,2296,752.0,18.0,805,810.0,5.0,0,0,0
5,2015,12,1,2,AA,2020,N977UY,PHX,CLT,15,...,182.0,1773,523.0,6.0,601,529.0,-32.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479215,2015,12,31,4,UA,668,N66848,LAS,IAD,2359,...,203.0,2065,634.0,18.0,721,652.0,-29.0,0,0,0
479216,2015,12,31,4,UA,671,N33209,PDX,IAH,2359,...,214.0,1825,536.0,9.0,604,545.0,-19.0,0,0,0
479217,2015,12,31,4,UA,1849,N497UA,LAS,EWR,2359,...,231.0,2227,703.0,15.0,742,718.0,-24.0,0,0,0
479218,2015,12,31,4,AA,2342,N3DLAA,DEN,MIA,2359,...,190.0,1709,519.0,6.0,543,525.0,-18.0,0,0,0


In [12]:
# Merge DateTime to one column
flights_df['DATE'] = pd.to_datetime(flights_df[['YEAR','MONTH','DAY']])
flights_df = flights_df.drop(columns=['YEAR','MONTH','DAY'])
flights_df.head()

,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,...,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,DATE
0,2,AA,1230,N3KSAA,SEA,DFW,5,9.0,4.0,10.0,...,1660,533.0,5.0,555,538.0,-17.0,0,0,0,2015-12-01
2,2,AA,2406,N850AA,LAX,DFW,10,7.0,-3.0,17.0,...,1235,445.0,8.0,501,453.0,-8.0,0,0,0,2015-12-01
3,2,UA,680,N38473,SFO,ORD,14,10.0,-4.0,11.0,...,1846,557.0,5.0,632,602.0,-30.0,0,0,0,2015-12-01
4,2,AA,883,N199UW,SFO,CLT,15,13.0,-2.0,15.0,...,2296,752.0,18.0,805,810.0,5.0,0,0,0,2015-12-01
5,2,AA,2020,N977UY,PHX,CLT,15,11.0,-4.0,10.0,...,1773,523.0,6.0,601,529.0,-32.0,0,0,0,2015-12-01


In [13]:
# Saturday has the least flights
# Thursday has the most flights
flights_df.groupby('DAY_OF_WEEK').count()

,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,...,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,DATE
DAY_OF_WEEK,,,,,,,,,,,,,,,,,,,,,
1,31427,31427,31427,31427,31427,31427,31427,31427,31427,31427,...,31427,31427,31427,31427,31427,31427,31427,31427,31427,31427
2,38710,38710,38710,38710,38710,38710,38710,38710,38710,38710,...,38710,38710,38710,38710,38710,38710,38710,38710,38710,38710
3,39426,39426,39426,39426,39426,39426,39426,39426,39426,39426,...,39426,39426,39426,39426,39426,39426,39426,39426,39426,39426
4,36756,36756,36756,36756,36756,36756,36756,36756,36756,36756,...,36756,36756,36756,36756,36756,36756,36756,36756,36756,36756
5,30190,30190,30190,30190,30190,30190,30190,30190,30190,30190,...,30190,30190,30190,30190,30190,30190,30190,30190,30190,30190
6,26337,26337,26337,26337,26337,26337,26337,26337,26337,26337,...,26337,26337,26337,26337,26337,26337,26337,26337,26337,26337
7,30169,30169,30169,30169,30169,30169,30169,30169,30169,30169,...,30169,30169,30169,30169,30169,30169,30169,30169,30169,30169


In [14]:
# Change Day of week from num to string
flights_df.loc[flights_df["DAY_OF_WEEK"] == 1, "DAY_OF_WEEK"] = "Monday"
flights_df.loc[flights_df["DAY_OF_WEEK"] == 2, "DAY_OF_WEEK"] = "Tuesday"
flights_df.loc[flights_df["DAY_OF_WEEK"] == 3, "DAY_OF_WEEK"] = "Wednesday"
flights_df.loc[flights_df["DAY_OF_WEEK"] == 4, "DAY_OF_WEEK"] = "Thursday"
flights_df.loc[flights_df["DAY_OF_WEEK"] == 5, "DAY_OF_WEEK"] = "Friday"
flights_df.loc[flights_df["DAY_OF_WEEK"] == 6, "DAY_OF_WEEK"] = "Saturday"
flights_df.loc[flights_df["DAY_OF_WEEK"] == 7, "DAY_OF_WEEK"] = "Sunday"

### Merge with airline and airport df

In [15]:
flights_air = pd.merge(flights_df,airline_df,left_on='AIRLINE',right_on='AIRLINE_ID',how='left')
flights_air = flights_air.drop(columns=['AIRLINE_ID'])
flights_air = flights_air.rename(columns={'AIRLINE_x': 'AIRLINE_ID', 'AIRLINE_y': 'AIRLINE'})
flights_air

,DAY_OF_WEEK,AIRLINE_ID,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,...,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,DATE,AIRLINE
0,Tuesday,AA,1230,N3KSAA,SEA,DFW,5,9.0,4.0,10.0,...,533.0,5.0,555,538.0,-17.0,0,0,0,2015-12-01,American Airlines Inc.
1,Tuesday,AA,2406,N850AA,LAX,DFW,10,7.0,-3.0,17.0,...,445.0,8.0,501,453.0,-8.0,0,0,0,2015-12-01,American Airlines Inc.
2,Tuesday,UA,680,N38473,SFO,ORD,14,10.0,-4.0,11.0,...,557.0,5.0,632,602.0,-30.0,0,0,0,2015-12-01,United Air Lines Inc.
3,Tuesday,AA,883,N199UW,SFO,CLT,15,13.0,-2.0,15.0,...,752.0,18.0,805,810.0,5.0,0,0,0,2015-12-01,American Airlines Inc.
4,Tuesday,AA,2020,N977UY,PHX,CLT,15,11.0,-4.0,10.0,...,523.0,6.0,601,529.0,-32.0,0,0,0,2015-12-01,American Airlines Inc.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233010,Thursday,UA,668,N66848,LAS,IAD,2359,2356.0,-3.0,15.0,...,634.0,18.0,721,652.0,-29.0,0,0,0,2015-12-31,United Air Lines Inc.
233011,Thursday,UA,671,N33209,PDX,IAH,2359,2352.0,-7.0,10.0,...,536.0,9.0,604,545.0,-19.0,0,0,0,2015-12-31,United Air Lines Inc.
233012,Thursday,UA,1849,N497UA,LAS,EWR,2359,2357.0,-2.0,15.0,...,703.0,15.0,742,718.0,-24.0,0,0,0,2015-12-31,United Air Lines Inc.
233013,Thursday,AA,2342,N3DLAA,DEN,MIA,2359,2351.0,-8.0,18.0,...,519.0,6.0,543,525.0,-18.0,0,0,0,2015-12-31,American Airlines Inc.


### Delayed Flights

In [16]:
# The total df of delay flights
arrival_Delay = flights_air.loc[flights_air['ARRIVAL_DELAY'] > 0].reset_index(drop=True)  # negative value means arrive earlier
departure_Delay=flights_air.loc[flights_air['DEPARTURE_DELAY']> 0].reset_index(drop=True)

#  for dep_Delay: 2125618 rows
#  for arr_Delay: 2086896 rows

In [17]:
# Both arrival and departure are delayed
delay_Flights = flights_air.loc[(flights_air['ARRIVAL_DELAY'] > 0) & (flights_air['DEPARTURE_DELAY'] > 0)].reset_index(drop=True)
delay_Flights  
# 1508147 total

,DAY_OF_WEEK,AIRLINE_ID,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,...,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,DATE,AIRLINE
0,Tuesday,AA,481,N165US,PHX,MSP,105,135.0,30.0,15.0,...,522.0,7.0,502,529.0,27.0,0,0,0,2015-12-01,American Airlines Inc.
1,Tuesday,UA,1162,N39418,BQN,EWR,245,318.0,33.0,10.0,...,548.0,6.0,545,554.0,9.0,0,0,0,2015-12-01,United Air Lines Inc.
2,Tuesday,AA,82,N3CVAA,SFO,ORD,500,507.0,7.0,15.0,...,1106.0,12.0,1117,1118.0,1.0,0,0,0,2015-12-01,American Airlines Inc.
3,Tuesday,AA,1171,N3AKAA,ORD,PHX,500,542.0,42.0,9.0,...,816.0,3.0,755,819.0,24.0,0,0,0,2015-12-01,American Airlines Inc.
4,Tuesday,AA,1310,N550AA,MSP,ORD,500,501.0,1.0,42.0,...,642.0,9.0,646,651.0,5.0,0,0,0,2015-12-01,American Airlines Inc.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71474,Thursday,AA,14,N858AA,OGG,LAX,2246,2.0,76.0,12.0,...,644.0,9.0,558,653.0,55.0,0,0,0,2015-12-31,American Airlines Inc.
71475,Thursday,UA,1695,N73278,SEA,EWR,2309,2321.0,12.0,14.0,...,718.0,11.0,710,729.0,19.0,0,0,0,2015-12-31,United Air Lines Inc.
71476,Thursday,AA,645,N908AW,OGG,PHX,2330,240.0,190.0,65.0,...,1149.0,8.0,803,1157.0,234.0,0,0,0,2015-12-31,American Airlines Inc.
71477,Thursday,AA,217,N3BAAA,LAS,MIA,2347,2350.0,3.0,19.0,...,706.0,14.0,714,720.0,6.0,0,0,0,2015-12-31,American Airlines Inc.


In [18]:
# Further Narrow down the dataframe
delay_Flights = delay_Flights[['AIRLINE_ID','AIRLINE','DATE','DAY_OF_WEEK','TAIL_NUMBER','FLIGHT_NUMBER','ORIGIN_AIRPORT',
                                     'DESTINATION_AIRPORT','DEPARTURE_DELAY','ARRIVAL_DELAY']]
delay_Flights.head(5)
# We are going to merge this dataset with tweets below.

,AIRLINE_ID,AIRLINE,DATE,DAY_OF_WEEK,TAIL_NUMBER,FLIGHT_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_DELAY,ARRIVAL_DELAY
0,AA,American Airlines Inc.,2015-12-01,Tuesday,N165US,481,PHX,MSP,30.0,27.0
1,UA,United Air Lines Inc.,2015-12-01,Tuesday,N39418,1162,BQN,EWR,33.0,9.0
2,AA,American Airlines Inc.,2015-12-01,Tuesday,N3CVAA,82,SFO,ORD,7.0,1.0
3,AA,American Airlines Inc.,2015-12-01,Tuesday,N3AKAA,1171,ORD,PHX,42.0,24.0
4,AA,American Airlines Inc.,2015-12-01,Tuesday,N550AA,1310,MSP,ORD,1.0,5.0


### Cancelled Flights

In [19]:
# Find out the values within the column
flights_air['CANCELLED'].value_counts()

0    229285
1      3730
Name: CANCELLED, dtype: int64

In [20]:
cancelled_flights = flights_air.loc[flights_air['CANCELLED']==1].reset_index(drop=True)

In [21]:
# Count total number of cancelled flights
cancelled_flights.count()['CANCELLED']

3730

In [22]:
# Remove irrelevant columns 
cancelled_flights = cancelled_flights[['AIRLINE_ID','AIRLINE','DATE','DAY_OF_WEEK','TAIL_NUMBER','FLIGHT_NUMBER','ORIGIN_AIRPORT',
                                     'DESTINATION_AIRPORT','DEPARTURE_DELAY','ARRIVAL_DELAY']]
cancelled_flights.head(5)

,AIRLINE_ID,AIRLINE,DATE,DAY_OF_WEEK,TAIL_NUMBER,FLIGHT_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_DELAY,ARRIVAL_DELAY
0,WN,Southwest Airlines Co.,2015-12-01,Tuesday,N915WN,1294,LAX,PHX,0.0,0.0
1,WN,Southwest Airlines Co.,2015-12-01,Tuesday,N494WN,3197,LAS,PHX,0.0,0.0
2,AA,American Airlines Inc.,2015-12-01,Tuesday,0,1764,FLL,DCA,0.0,0.0
3,AA,American Airlines Inc.,2015-12-01,Tuesday,N873AA,2462,DFW,LAX,0.0,0.0
4,UA,United Air Lines Inc.,2015-12-01,Tuesday,0,1728,OGG,SFO,0.0,0.0


# 4. Tweets data

In [23]:
tweets_file = "./Resources/Tweets.csv"
tweets_df = pd.read_csv(tweets_file)
tweets_df.head(5)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [24]:
tweets_df = tweets_df[['tweet_id','airline_sentiment','negativereason','airline','tweet_created']]
tweets_df.head(5)

,tweet_id,airline_sentiment,negativereason,airline,tweet_created
0,570306133677760513,neutral,NaN,Virgin America,2015-02-24 11:35:52 -0800
1,570301130888122368,positive,NaN,Virgin America,2015-02-24 11:15:59 -0800
2,570301083672813571,neutral,NaN,Virgin America,2015-02-24 11:15:48 -0800
3,570301031407624196,negative,Bad Flight,Virgin America,2015-02-24 11:15:36 -0800
4,570300817074462722,negative,Can't Tell,Virgin America,2015-02-24 11:14:45 -0800


In [25]:
tweets_df['airline'].value_counts()

United            3822
US Airways        2913
American          2759
Southwest         2420
Delta             2222
Virgin America     504
Name: airline, dtype: int64

In [26]:
# Update the airline Name so it matches the flights_df
tweets_df.loc[tweets_df["airline"] == "American", "AIRLINE"] = "American Airlines Inc."
tweets_df.loc[tweets_df["airline"] == "Delta",    "AIRLINE"] = "Delta Air Lines Inc."
tweets_df.loc[tweets_df["airline"] == "Southwest","AIRLINE"] = "Southwest Airlines Co."
tweets_df.loc[tweets_df["airline"] == "US Airways","AIRLINE"] = "US Airways Inc."
tweets_df.loc[tweets_df["airline"] == "United", "AIRLINE"] = "United Air Lines Inc."
tweets_df.loc[tweets_df["airline"] == "Virgin America", "AIRLINE"] = "Virgin America"

In [ ]:
# # Clean the tweet_created column to get the Date
# tweets_df['DATE'] = pd.to_datetime(tweets_df['tweet_created'])
# tweets_df['DATE']=pd.DatetimeIndex(tweets_df['DATE']).date
# tweets_df = tweets_df.drop(columns=['tweet_created','airline'])

In [ ]:
# Another Method:
# tweets_df['DATE'] = tweets_df['tweet_created'].str[0:10]
# tweets_df['DATE'] = pd.to_datetime(tweets_df['DATE'])
# tweets_df.head()
# tweets_df = tweets_df.drop(columns=['tweet_created'])
# tweets_df.dtypes

In [27]:
# replace NaN with "No comments"
tweets_df['negativereason'] = tweets_df['negativereason'].fillna('No comments')
tweets_df

,tweet_id,airline_sentiment,negativereason,airline,tweet_created,AIRLINE
0,570306133677760513,neutral,No comments,Virgin America,2015-02-24 11:35:52 -0800,Virgin America
1,570301130888122368,positive,No comments,Virgin America,2015-02-24 11:15:59 -0800,Virgin America
2,570301083672813571,neutral,No comments,Virgin America,2015-02-24 11:15:48 -0800,Virgin America
3,570301031407624196,negative,Bad Flight,Virgin America,2015-02-24 11:15:36 -0800,Virgin America
4,570300817074462722,negative,Can't Tell,Virgin America,2015-02-24 11:14:45 -0800,Virgin America
...,...,...,...,...,...,...
14635,569587686496825344,positive,No comments,American,2015-02-22 12:01:01 -0800,American Airlines Inc.
14636,569587371693355008,negative,Customer Service Issue,American,2015-02-22 11:59:46 -0800,American Airlines Inc.
14637,569587242672398336,neutral,No comments,American,2015-02-22 11:59:15 -0800,American Airlines Inc.
14638,569587188687634433,negative,Customer Service Issue,American,2015-02-22 11:59:02 -0800,American Airlines Inc.


In [28]:
tweets_df['AIRLINE'].value_counts()

United Air Lines Inc.     3822
US Airways Inc.           2913
American Airlines Inc.    2759
Southwest Airlines Co.    2420
Delta Air Lines Inc.      2222
Virgin America             504
Name: AIRLINE, dtype: int64

In [29]:
tweets_df['airline_sentiment'].value_counts()

negative    9178
neutral     3099
positive    2363
Name: airline_sentiment, dtype: int64

In [30]:
tweets_df.groupby(['AIRLINE','airline_sentiment']).count()
# Apparently, United AirLines has the most negative reviews
# Virgin America has the least nagative reviews

tweet_id  negativereason  airline  \
AIRLINE                airline_sentiment                                      
American Airlines Inc. negative               1960            1960     1960   
                       neutral                 463             463      463   
                       positive                336             336      336   
Delta Air Lines Inc.   negative                955             955      955   
                       neutral                 723             723      723   
                       positive                544             544      544   
Southwest Airlines Co. negative               1186            1186     1186   
                       neutral                 664             664      664   
                       positive                570             570      570   
US Airways Inc.        negative               2263            2263     2263   
                       neutral                 381             381      381   
                       positive                269             269      269   
United Air Lines Inc.  negative               2633            2633     2633   
                       neutral                 697             697      697   
                       positive                492             492      492   
Virgin America         negative                181             181      181   
                       neutral                 171             171      171   
                       positive                152             152      152   

                                          tweet_created  
AIRLINE                airline_sentiment                 
American Airlines Inc. negative                    1960  
                       neutral                      463  
                       positive                     336  
Delta Air Lines Inc.   negative                     955  
                       neutral                      723  
                       positive                     544  
Southwest Airlines Co. negative                    1186  
                       neutral                      664  
                       positive                     570  
US Airways Inc.        negative                    2263  
                       neutral                      381  
                       positive                     269  
United Air Lines Inc.  negative                    2633  
                       neutral                      697  
                       positive                     492  
Virgin America         negative                     181  
                       neutral                      171  
                       positive                     152

In [ ]:
# If we want to change NaN valule to 0
# tweets_df.loc[tweets_df['set_of_numbers'].isnull(),'value_is_NaN'] = 'Yes'
# tweets_df.loc[tweets_df['set_of_numbers'].notnull(),'value_is_NaN'] = 'No'

# count_nan = tweets_df.loc[tweets_df['value_is_NaN']=='Yes'].count()
# print (count_nan)

## Connect to MongoDB

In [31]:
#import pymysql
#pymysql.install_as_MySQLdb()
import pymongo
#import MySQLdb

In [32]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [33]:
db = client.flights_db

In [34]:
#airport code data
for index,row in airports_df.iterrows():
    #print(row)
    db.airports_df.insert_one(dict(row))

In [35]:
#tweet data table
for index,row in tweets_df.iterrows():
    #print(row)
    db.tweets_df.insert_one(dict(row))

In [36]:
#airline and flight delay tables merged
for index,row in flights_air.iterrows():
    #print(row)
    db.flight.insert_one(dict(row))
    #break

In [ ]:
for item in db.flight.find():
    print(item)

In [ ]:
# db.flight.find()